# Final Project Workbook
### BIOF 309 Spring 2020 

**Author:** Ramita Karra <br>
**Last edited:** 04-23-2020

In [10]:
%pwd
%load_ext autoreload
%autoreload 2

### Initialize and set up package files

#### Create new a directory and directory structure for the package (slightly modified code from class)

In [ ]:
%%writefile initialize.py

from pathlib import Path
import os
import shutil 

def create_package_dir(package_name='BIOF309_RDK'):
    """ This function creates a new directory within the current directory using the passed input string 
    package_name. If a directory with the same name as package_name already exists, it deletes the old 
    directory."""
    
    start_dir = Path.cwd()
    print(f"Starting in {start_dir}")

    if start_dir.name == package_name:
        os.chdir(start_dir.parent)
        package_dir = start_dir
    else:
        package_dir = start_dir / package_name
    
    if package_dir.exists():
        print("Removing old directory...")
        shutil.rmtree(package_dir)

    print(f"Creating {package_dir}...")
    package_dir.mkdir()
    print(f"The current working directory is now {package_dir}")
    os.chdir(package_dir)
    
def create_package_str(package_name='EHT_RDK'):
    """ This function creates a new package structure within the current directory. It creates a new directory
    to house code, with the passed input string package_name, as well as blank files for 'init.py' within the 
    new directory, setup.py, License and Readme files."""
    
    Path('tests').mkdir()
    python_dir = Path(package_name)
    python_dir.mkdir()
    (python_dir / '__init__.py').touch()
    Path('setup.py').touch()
    Path('LICENSE').touch()
    Path('README.md').touch()

In [ ]:
from initialize import create_package_dir, create_package_str

create_package_dir('BIOF309_RDK')
create_package_str('EHT_RDK')

#### Add metadata and installation details (slightly modified code from class)

In [ ]:
%%writefile setup.py

# Metadata and installation details for the EHT_RDK package

import setuptools

with open("README.md", "r") as fh:
    long_description = fh.read()

setuptools.setup(
    name="EHT_RDK", 
    version="0.0.1",
    author="Ramita D. Karra",
    author_email="ramita.karra@nih.gov",
    description="A package for processing Expansion Hunter Targeted (EHT) repeat data",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.com/pypa/packaging_demo",
    packages=setuptools.find_packages(),
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
    ],
    python_requires='>=3.6',

)

#### Modify README

In [ ]:
%%writefile README.md

# EHT_RDK
## Package Description
<br>
This aim of this package is to process raw tab-delimited output returned from the ExpansionHunter-Targeted 
software tool, used for making sequence-graph-based predictions of repeat lengths for known genetic repeat loci. 
The ultimate goal is to clean, compile, and process data for many different loci into a summary table, and to 
provide visualizations pertinent to the functional relevance of this data (i.e. number of samples containing 
repeat numbers above the pathogenic threshold for each gene).  

More information on ExpansionHunter can be found [here](https://academic.oup.com/bioinformatics/article/35/22/4754/5499079). 

#### Write license (from class)

In [ ]:
%%writefile LICENSE

Copyright (c) 2018 The Python Packaging Authority

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

### Input files

#### Create a list of file paths, consisting of all files in the directory containing raw data

In [ ]:
%%writefile EHT_RDK/input_files.py

import os
import glob
import pandas as pd

def create_input_list(directory_name):
    file_list = []

    # Check to make sure that only '.txt' files are being appended to list
    for filename in os.listdir(directory_name):
        if filename.endswith('.txt'):
            file_list.append(filename)
        else:
            print('Found non .txt file in directory: ' + filename)
            
    return(file_list)

In [ ]:
from input_files import create_input_list

# Pass directory containing raw ExpansionHunter - Targeted data
files_to_import = create_input_list('/Users/dewanr2/Documents/Ramitas_Docs/NIH_Classes/BIOF309/ExpansionHunterTargeted')

In [ ]:
display(files_to_import)

#### EXPLORATORY DATA ANALYSIS

In [ ]:
# Change into directory containing raw data

directory_name = '/Users/dewanr2/Documents/Ramitas_Docs/NIH_Classes/BIOF309/ExpansionHunterTargeted'
os.chdir(directory_name)

In [ ]:
%pwd

In [ ]:
# Import first file in file_list 
test_df = pd.read_csv(file_list[0], sep='\t')

# Examine columns
print(test_df.columns)

In [ ]:
# Create 'min' and 'max' columns for minimum and maximum allele repeat values respectively
test_df['min'] = test_df[['REPCN:ATXN7_GCC_allele1','REPCN:ATXN7_GCC_allele2']].min(axis=1)
test_df['max'] = test_df[['REPCN:ATXN7_GCC_allele1','REPCN:ATXN7_GCC_allele2']].max(axis=1)

# Select only desired columns
test_df = test_df[['SampleID','chr','pos','min','max']]

print(test_df.columns)
print(test_df.info())

*EDA suggests that when importing dataframes, need to apply the following:*
- evaluate data to create 'min' and 'max' columns
- rename 'max' column with gene name
- select only desired columns: 'SampleID','chr','pos','max'

*EDA did not show any null entries for this df, but need to import with default null value in case all genes were not evaluated for all samples*

#### Import all files from list of filepaths created earlier

In [ ]:
%%writefile -a input_files.py

def create_dataframe_list(file_list, dir_name):
    """This function creates a list of dataframes from the passed input list containing filenames. The 
    'SampleID','chr',and 'pos' columns are retained from the original dataframe, and a new column is created
    for the maximum repeats on either allele (column heading is the gene name).""" 

    df_list = []

    for filename in file_list:
    
        # Get gene name, avoid filenames without expected suffix/prefix
        if filename.startswith("ExpansionHunterTargeted.ftd"):
            gene = filename.replace("ExpansionHunterTargeted.ftd.","", 1)
            if gene.endswith(".txt"):
                gene = gene.replace(".txt","",1)
            else:
                print("filename does not contain suffix")
        else:
            print("filename does not contain prefix")
    
        # Import and format df
        df_temp = pd.read_csv(os.path.join(dir_name, filename), sep='\t')
        df_temp.rename(columns={ df_temp.columns[6]: "allele1" }, inplace = True)
        df_temp.rename(columns={ df_temp.columns[7]: "allele2" }, inplace = True)
        df_temp['min'] = df_temp[['allele1','allele2']].min(axis=1)
        df_temp[gene] = df_temp[['allele1','allele2']].max(axis=1)
        df_temp = df_temp[['SampleID','chr','pos',gene]]
        df_list.append(df_temp)
        
    return(df_list)

In [ ]:
%cd /Users/dewanr2/Documents/GitHub/project_spring_2020/project_spring_2020/BIOF309_RDK/EHT_RDK

In [ ]:
from input_files import create_input_list, create_dataframe_list

# Pass list of filenames created earlier, to create list of cleaned dataframes
df_list = create_dataframe_list(files_to_import, '/Users/dewanr2/Documents/Ramitas_Docs/NIH_Classes/BIOF309/ExpansionHunterTargeted/')

In [ ]:
display(df_list[0].head())

### Create dictionary of gene info

In [ ]:
%pwd

In [ ]:
%%writefile process_gene_dfs.py

def create_gene_dict(df_list):
    
    """This function creates a dictionary with keys as genes, subdictionaries as key:value pairs for "chr", 
    "pos". It uses a list of dataframes passed in as an input parameter..""" 
    
    gene_dict = {}

    for df in df_list:
        # Get gene name
        gene = df.columns[3]

        # Get chromosome
        chr_num = df.iloc[0,1]
        chr_num = chr_num.replace("chr","",1) # Remove "chr" prefix

        # Get gene position
        pos = df.iloc[0,2]

        # Edit dictionary
        gene_dict.update( {gene : {"chr":chr_num, "pos":pos}} )
    
    return(gene_dict)

In [ ]:
from process_gene_dfs import create_gene_dict

gene_dict = create_gene_dict(df_list)

### Merge gene-specific dataframes into master dataframe

In [ ]:
%%writefile -a process_gene_dfs.py

def merge_dfs(old_df_list):
    """This function merges the list of dataframes passed in as an input parameter, by first re-indexing the 
    dataframes by SampleID, and then merging based on SampleID""" 
    
    # Change sampleID to index in dfs
    df_reindex_list = []

    for df in old_df_list:
        column_names = list(df.columns)
        temp_reindex_df = df.set_index(df['SampleID'])
        temp_reindex_df = temp_reindex_df[column_names[1:]]
        df_reindex_list.append(temp_reindex_df)

    # Create a list of dfs to merge
    to_merge = []

    for df in df_reindex_list:
        column_names = list(df.columns)
        temp_df = df[column_names[2]]
        to_merge.append(temp_df) 

    # Merge dfs on sampleID to create master df of all gene max repeat lengths
    # Use 'outer' join to keep all samples, including those that do not contain information for all genes
    master_df = pd.concat(to_merge, axis = 1, join='outer', sort=False)
    
    return(master_df)

In [ ]:
from process_gene_dfs import merge_dfs

master_df = merge_dfs(df_list)

### Further analysis: create summary df of samples with repeats above pathogenic range

In [ ]:
%%writefile further_analyses.py

import pandas as pd

def create_summary_df(genes_of_interest, pathogenic_bounds, gene_dict, master_df, export=False):
    """This function creates a summary table of the number of pathogenic samples for each of the genes of interest.
    It accepts a list of genes of interest, a list of the pathogenic bounds for those genes, the previously
    generated gene dictionary, and the master dataframe of all genes and repeat info as input parameters. It
    exports the dataframe if the input parameter 'export' is specified as 'True' by the user."""

    # Add pathogenic range for 10 desired genes to dictionary
    for gene in genes_of_interest:
        gene_dict[gene].update( {"pathogenic_bound":pathogenic_bounds[genes_of_interest.index(gene)]} )

    # Count number of samples with repeats greater than or equal to pathogenic_bound
    # Append to dictionary
    for gene in genes_of_interest:
        path_count = len(master_df[master_df[gene] >= gene_dict[gene]['pathogenic_bound']])
        gene_dict[gene].update( {"pathogenic_count":path_count} )
        
    # Create new gene dictionary for desired genes only
    new_gene_dict = { gene_key: gene_dict[gene_key] for gene_key in genes_of_interest }

    # Create dataframe for desired genes using new_gene_dict dictionary
    summary_df = pd.DataFrame.from_dict(new_gene_dict, orient='index')
    summary_df.columns = ['Chromosome','Position','Pathogenic Bound','Number of Pathogenic Samples']
    summary_df.index.name = 'Gene'
    
    # Export summary dataframe
    if export==True:
        summary_df.to_csv('Pathoenic_Repeat_Data_Summary', index=True)
        
    return(summary_df)

In [ ]:
from further_analyses import create_summary_df

gene_list = ['AR','ATN1','ATXN1','ATXN3','C9ORF72','DMPK','FMR1','FXN.GAA','HTT.CAG','PHOX2B']
bounds_list = [37, 48, 39, 52, 30, 50, 200, 66, 40, 25]

summary_df = create_summary_df(gene_list, bounds_list, gene_dict, master_df)

In [ ]:
display(summary_df)

### Further analysis: create plot of number of pathogenic samples for each gene of interest

In [ ]:
%%writefile -a further_analyses.py

def plot_pathogenic(summary_df, export=False):
    
    """This function plots the summary pathogenic repeat counts using the summary_df input parameter"""
    
    import matplotlib.pyplot as plt
    
    # Plot pathogenic samples for each gene of interest
    summary_df['Number of Pathogenic Samples'].plot(kind='bar')
    plt.minorticks_on()
    plt.title('Summary of Pathogenic Repeat Data')
    plt.xlabel('Genes')
    plt.ylabel('Number of Pathogenic Samples')

    plt.show()
    
    # Save plot to file 
    if export==True:
        plt.savefig('Pathogenic_Repeat_Data_Summary_Plot.png')

In [ ]:
from further_analyses import plot_pathogenic

plot_pathogenic(summary_df)

### Test functions

In [1]:
%pwd

'/Users/dewanr2/Documents/GitHub/project_spring_2020/project_spring_2020'

In [27]:
%%writefile test_input.py

from EHT_RDK.input_files import *

def test_input_list():
    directory_name = "/Users/dewanr2/Documents/Ramitas_Docs/NIH_Classes/BIOF309/Input_test"
    obs = create_input_list(directory_name)
    exp = []
    assert obs == exp
    
def test_df_list():
    dir_name = "/Users/dewanr2/Documents/Ramitas_Docs/NIH_Classes/BIOF309/create_df_test"
    file_list = ['AFF2.txt']
    obs = create_dataframe_list(file_list, dir_name)
    exp = NotImplemented
    assert obs == exp

Writing test_input.py


In [20]:
%cd ..

/Users/dewanr2/Documents/GitHub/project_spring_2020/project_spring_2020/BIOF309_RDK


In [22]:
!pip install -e .

Obtaining file:///Users/dewanr2/Documents/GitHub/project_spring_2020/project_spring_2020/BIOF309_RDK
  Running setup.py develop for EHT-RDK


In [28]:
!pytest ./tests/

============================= test session starts ==============================
platform darwin -- Python 3.7.1, pytest-4.0.2, py-1.7.0, pluggy-0.8.0
rootdir: /Users/dewanr2/Documents/GitHub/project_spring_2020/project_spring_2020/BIOF309_RDK, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.2.0, arraydiff-0.3
collected 2 items                                                              

tests/test_input.py ..                                                   [100%]

=============================== warnings summary ===============================
/Users/dewanr2/anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/inference.py:6
  /Users/dewanr2/anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/inference.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
    from collections import Iterable

/Users/dewanr2/anaconda3/lib/python3.7/site-packages/pandas/cor